# 变分自编码器

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.utils import save_image

In [57]:
# 定义超参数
image_size = 28*28
h_dim = 400
z_dim = 20
num_epoches = 10
batch_size = 128
learning_rate = 1e-3

torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [58]:
data_train = datasets.MNIST("./data", train=True, download=False, 
                            transform=transforms.ToTensor())
data_test = datasets.MNIST("./data", train=False, download=False, 
                            transform=transforms.ToTensor())

In [59]:
data_train_loader = DataLoader(dataset=data_train, batch_size=batch_size, 
                               shuffle=True)
data_test_loader = DataLoader(dataset=data_test, batch_size=batch_size, 
                               shuffle=True)

In [60]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc21 = nn.Linear(h_dim, z_dim)
        self.fc22 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        self.fc4 = nn.Linear(h_dim, image_size)
    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3)) 
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, image_size))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [61]:
model =  VAE().to(device)

In [62]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [63]:
def loss_func(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum') # 这里假设每个像素服从伯努利分布，如果是实值，可假设服从高斯分布
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) # q_\phi(z|x)与p(z)先验的KLD
    return BCE + KLD

In [64]:
log_interval = 10 # 每10个batch输出一次损失
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(data_train_loader):
        data = data.to(device)
        recon_x, mu, logvar = model(data)
        loss = loss_func(recon_x, data, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print("Trian Epoch:{} [{}/{} ({:.0f}%)] \tLoss:{:.6f}".format(
            epoch, batch_idx*len(data), len(data_train_loader.dataset), 
                100. * batch_idx / len(data_train_loader), loss.item() / len(data)))
        
        print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(data_train_loader.dataset)))

In [65]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(data_test_loader):
            data = data.to(device)
            recon_x, mu, logvar = model(data)
            test_loss += loss_func(recon_x, data, mu, logvar).item()
            # 比较第一个batch中真实图像与生成图像
            if i == 0: 
                n = min(len(data), 8) # 比较的图片数量
                comparison = torch.cat([data[:n], recon_x.view(-1, 1, 28, 28)[:n]])
                save_image(comparison.cpu(), "./results/reconstruction_" + str(epoch) + ".png", nrow=n)
            test_loss /= len(data_test_loader.dataset)
            print('====> Test set loss: {:.4f}'.format(test_loss))

In [66]:
def main():
    for epoch in range(1, num_epoches + 1):
        train(epoch)
        test(epoch)

In [67]:
main()

Trian Epoch:1 [0/60000 (0%)] 	Loss:550.800720
====> Epoch: 1 Average loss: 1.1750
====> Epoch: 1 Average loss: 2.2945
====> Epoch: 1 Average loss: 3.3600
====> Epoch: 1 Average loss: 4.3720
====> Epoch: 1 Average loss: 5.3218
====> Epoch: 1 Average loss: 6.2023
====> Epoch: 1 Average loss: 7.0177
====> Epoch: 1 Average loss: 7.7652
====> Epoch: 1 Average loss: 8.4426
====> Epoch: 1 Average loss: 9.0891
Trian Epoch:1 [1280/60000 (2%)] 	Loss:322.319305
====> Epoch: 1 Average loss: 9.7767
====> Epoch: 1 Average loss: 10.4151
====> Epoch: 1 Average loss: 11.0203
====> Epoch: 1 Average loss: 11.5700
====> Epoch: 1 Average loss: 12.1073
====> Epoch: 1 Average loss: 12.6336
====> Epoch: 1 Average loss: 13.1633
====> Epoch: 1 Average loss: 13.6883
====> Epoch: 1 Average loss: 14.1965
====> Epoch: 1 Average loss: 14.7063
Trian Epoch:1 [2560/60000 (4%)] 	Loss:238.075073
====> Epoch: 1 Average loss: 15.2142
====> Epoch: 1 Average loss: 15.7193
====> Epoch: 1 Average loss: 16.2286
====> Epoch: 1 A

====> Epoch: 1 Average loss: 83.8572
====> Epoch: 1 Average loss: 84.1889
====> Epoch: 1 Average loss: 84.5172
====> Epoch: 1 Average loss: 84.8591
====> Epoch: 1 Average loss: 85.1906
====> Epoch: 1 Average loss: 85.5223
Trian Epoch:1 [25600/60000 (43%)] 	Loss:148.721207
====> Epoch: 1 Average loss: 85.8396
====> Epoch: 1 Average loss: 86.1754
====> Epoch: 1 Average loss: 86.5053
====> Epoch: 1 Average loss: 86.8385
====> Epoch: 1 Average loss: 87.1618
====> Epoch: 1 Average loss: 87.4893
====> Epoch: 1 Average loss: 87.8100
====> Epoch: 1 Average loss: 88.1292
====> Epoch: 1 Average loss: 88.4481
====> Epoch: 1 Average loss: 88.7934
Trian Epoch:1 [26880/60000 (45%)] 	Loss:152.023270
====> Epoch: 1 Average loss: 89.1177
====> Epoch: 1 Average loss: 89.4458
====> Epoch: 1 Average loss: 89.7801
====> Epoch: 1 Average loss: 90.1063
====> Epoch: 1 Average loss: 90.4393
====> Epoch: 1 Average loss: 90.7553
====> Epoch: 1 Average loss: 91.0734
====> Epoch: 1 Average loss: 91.3930
====> Epoc

====> Epoch: 1 Average loss: 142.4570
====> Epoch: 1 Average loss: 142.7345
====> Epoch: 1 Average loss: 143.0235
====> Epoch: 1 Average loss: 143.3054
====> Epoch: 1 Average loss: 143.5888
Trian Epoch:1 [49920/60000 (83%)] 	Loss:131.922089
====> Epoch: 1 Average loss: 143.8702
====> Epoch: 1 Average loss: 144.1498
====> Epoch: 1 Average loss: 144.4457
====> Epoch: 1 Average loss: 144.7289
====> Epoch: 1 Average loss: 145.0073
====> Epoch: 1 Average loss: 145.2945
====> Epoch: 1 Average loss: 145.5801
====> Epoch: 1 Average loss: 145.8571
====> Epoch: 1 Average loss: 146.1427
====> Epoch: 1 Average loss: 146.4253
Trian Epoch:1 [51200/60000 (85%)] 	Loss:131.970947
====> Epoch: 1 Average loss: 146.7069
====> Epoch: 1 Average loss: 146.9938
====> Epoch: 1 Average loss: 147.2729
====> Epoch: 1 Average loss: 147.5545
====> Epoch: 1 Average loss: 147.8457
====> Epoch: 1 Average loss: 148.1297
====> Epoch: 1 Average loss: 148.4243
====> Epoch: 1 Average loss: 148.7247
====> Epoch: 1 Average l

====> Epoch: 2 Average loss: 15.8217
====> Epoch: 2 Average loss: 16.0897
====> Epoch: 2 Average loss: 16.3592
Trian Epoch:2 [7680/60000 (13%)] 	Loss:127.505783
====> Epoch: 2 Average loss: 16.6312
====> Epoch: 2 Average loss: 16.8968
====> Epoch: 2 Average loss: 17.1619
====> Epoch: 2 Average loss: 17.4313
====> Epoch: 2 Average loss: 17.6997
====> Epoch: 2 Average loss: 17.9635
====> Epoch: 2 Average loss: 18.2358
====> Epoch: 2 Average loss: 18.5046
====> Epoch: 2 Average loss: 18.7724
====> Epoch: 2 Average loss: 19.0448
Trian Epoch:2 [8960/60000 (15%)] 	Loss:124.594376
====> Epoch: 2 Average loss: 19.3106
====> Epoch: 2 Average loss: 19.5722
====> Epoch: 2 Average loss: 19.8368
====> Epoch: 2 Average loss: 20.1101
====> Epoch: 2 Average loss: 20.3854
====> Epoch: 2 Average loss: 20.6628
====> Epoch: 2 Average loss: 20.9404
====> Epoch: 2 Average loss: 21.2146
====> Epoch: 2 Average loss: 21.4814
====> Epoch: 2 Average loss: 21.7631
Trian Epoch:2 [10240/60000 (17%)] 	Loss:123.97322

====> Epoch: 2 Average loss: 66.7729
====> Epoch: 2 Average loss: 67.0366
====> Epoch: 2 Average loss: 67.2934
====> Epoch: 2 Average loss: 67.5511
====> Epoch: 2 Average loss: 67.8134
====> Epoch: 2 Average loss: 68.0583
====> Epoch: 2 Average loss: 68.3162
====> Epoch: 2 Average loss: 68.5751
====> Epoch: 2 Average loss: 68.8301
====> Epoch: 2 Average loss: 69.0856
Trian Epoch:2 [33280/60000 (55%)] 	Loss:121.543991
====> Epoch: 2 Average loss: 69.3449
====> Epoch: 2 Average loss: 69.5985
====> Epoch: 2 Average loss: 69.8506
====> Epoch: 2 Average loss: 70.1222
====> Epoch: 2 Average loss: 70.3742
====> Epoch: 2 Average loss: 70.6336
====> Epoch: 2 Average loss: 70.8922
====> Epoch: 2 Average loss: 71.1461
====> Epoch: 2 Average loss: 71.4013
====> Epoch: 2 Average loss: 71.6616
Trian Epoch:2 [34560/60000 (58%)] 	Loss:120.920593
====> Epoch: 2 Average loss: 71.9196
====> Epoch: 2 Average loss: 72.1737
====> Epoch: 2 Average loss: 72.4298
====> Epoch: 2 Average loss: 72.6834
====> Epoc

...

====> Epoch: 9 Average loss: 21.6529
====> Epoch: 9 Average loss: 21.8895
====> Epoch: 9 Average loss: 22.1177
====> Epoch: 9 Average loss: 22.3478
====> Epoch: 9 Average loss: 22.5750
====> Epoch: 9 Average loss: 22.8052
Trian Epoch:9 [12800/60000 (21%)] 	Loss:111.325546
====> Epoch: 9 Average loss: 23.0427
====> Epoch: 9 Average loss: 23.2700
====> Epoch: 9 Average loss: 23.4928
====> Epoch: 9 Average loss: 23.7131
====> Epoch: 9 Average loss: 23.9389
====> Epoch: 9 Average loss: 24.1551
====> Epoch: 9 Average loss: 24.3939
====> Epoch: 9 Average loss: 24.6188
====> Epoch: 9 Average loss: 24.8379
====> Epoch: 9 Average loss: 25.0744
Trian Epoch:9 [14080/60000 (23%)] 	Loss:105.841934
====> Epoch: 9 Average loss: 25.3002
====> Epoch: 9 Average loss: 25.5235
====> Epoch: 9 Average loss: 25.7467
====> Epoch: 9 Average loss: 25.9784
====> Epoch: 9 Average loss: 26.2003
====> Epoch: 9 Average loss: 26.4263
====> Epoch: 9 Average loss: 26.6623
====> Epoch: 9 Average loss: 26.8898
====> Epoc

====> Epoch: 9 Average loss: 66.1405
Trian Epoch:9 [37120/60000 (62%)] 	Loss:103.792526
====> Epoch: 9 Average loss: 66.3619
====> Epoch: 9 Average loss: 66.5798
====> Epoch: 9 Average loss: 66.8050
====> Epoch: 9 Average loss: 67.0396
====> Epoch: 9 Average loss: 67.2667
====> Epoch: 9 Average loss: 67.4920
====> Epoch: 9 Average loss: 67.7213
====> Epoch: 9 Average loss: 67.9461
====> Epoch: 9 Average loss: 68.1756
====> Epoch: 9 Average loss: 68.4037
Trian Epoch:9 [38400/60000 (64%)] 	Loss:105.964615
====> Epoch: 9 Average loss: 68.6297
====> Epoch: 9 Average loss: 68.8520
====> Epoch: 9 Average loss: 69.0779
====> Epoch: 9 Average loss: 69.3044
====> Epoch: 9 Average loss: 69.5316
====> Epoch: 9 Average loss: 69.7616
====> Epoch: 9 Average loss: 69.9825
====> Epoch: 9 Average loss: 70.2026
====> Epoch: 9 Average loss: 70.4345
====> Epoch: 9 Average loss: 70.6625
Trian Epoch:9 [39680/60000 (66%)] 	Loss:107.341797
====> Epoch: 9 Average loss: 70.8915
====> Epoch: 9 Average loss: 71.1

====> Test set loss: 1.3438
====> Test set loss: 1.3382
====> Test set loss: 1.3754
====> Test set loss: 1.3238
====> Test set loss: 1.3665
====> Test set loss: 1.3517
====> Test set loss: 1.3751
====> Test set loss: 1.3170
====> Test set loss: 1.4351
====> Test set loss: 1.3748
====> Test set loss: 1.3233
====> Test set loss: 1.3729
====> Test set loss: 1.3539
====> Test set loss: 1.3483
====> Test set loss: 1.3971
====> Test set loss: 1.3233
====> Test set loss: 1.3575
====> Test set loss: 1.3274
====> Test set loss: 1.3491
====> Test set loss: 1.3504
====> Test set loss: 1.3631
====> Test set loss: 1.3909
====> Test set loss: 1.3545
====> Test set loss: 1.2824
====> Test set loss: 1.3496
====> Test set loss: 1.3605
====> Test set loss: 1.3736
====> Test set loss: 1.3571
====> Test set loss: 1.3698
====> Test set loss: 1.3670
====> Test set loss: 1.3717
====> Test set loss: 1.3516
====> Test set loss: 1.3555
====> Test set loss: 1.3619
====> Test set loss: 1.3287
====> Test set loss:

====> Epoch: 10 Average loss: 34.4436
====> Epoch: 10 Average loss: 34.6671
====> Epoch: 10 Average loss: 34.8929
====> Epoch: 10 Average loss: 35.1239
====> Epoch: 10 Average loss: 35.3448
====> Epoch: 10 Average loss: 35.5699
====> Epoch: 10 Average loss: 35.7950
====> Epoch: 10 Average loss: 36.0226
====> Epoch: 10 Average loss: 36.2526
Trian Epoch:10 [20480/60000 (34%)] 	Loss:108.821701
====> Epoch: 10 Average loss: 36.4847
====> Epoch: 10 Average loss: 36.7111
====> Epoch: 10 Average loss: 36.9453
====> Epoch: 10 Average loss: 37.1699
====> Epoch: 10 Average loss: 37.4041
====> Epoch: 10 Average loss: 37.6260
====> Epoch: 10 Average loss: 37.8616
====> Epoch: 10 Average loss: 38.0809
====> Epoch: 10 Average loss: 38.3071
====> Epoch: 10 Average loss: 38.5236
Trian Epoch:10 [21760/60000 (36%)] 	Loss:106.730499
====> Epoch: 10 Average loss: 38.7513
====> Epoch: 10 Average loss: 38.9793
====> Epoch: 10 Average loss: 39.2007
====> Epoch: 10 Average loss: 39.4294
====> Epoch: 10 Averag

====> Epoch: 10 Average loss: 77.3587
====> Epoch: 10 Average loss: 77.5783
====> Epoch: 10 Average loss: 77.8028
====> Epoch: 10 Average loss: 78.0240
====> Epoch: 10 Average loss: 78.2514
====> Epoch: 10 Average loss: 78.4780
====> Epoch: 10 Average loss: 78.7058
====> Epoch: 10 Average loss: 78.9294
====> Epoch: 10 Average loss: 79.1575
====> Epoch: 10 Average loss: 79.3800
Trian Epoch:10 [44800/60000 (75%)] 	Loss:106.296371
====> Epoch: 10 Average loss: 79.6068
====> Epoch: 10 Average loss: 79.8212
====> Epoch: 10 Average loss: 80.0528
====> Epoch: 10 Average loss: 80.2838
====> Epoch: 10 Average loss: 80.5112
====> Epoch: 10 Average loss: 80.7350
====> Epoch: 10 Average loss: 80.9583
====> Epoch: 10 Average loss: 81.1871
====> Epoch: 10 Average loss: 81.4082
====> Epoch: 10 Average loss: 81.6422
Trian Epoch:10 [46080/60000 (77%)] 	Loss:108.002563
====> Epoch: 10 Average loss: 81.8726
====> Epoch: 10 Average loss: 82.1028
====> Epoch: 10 Average loss: 82.3269
====> Epoch: 10 Averag

In [68]:
# 利用训练好的VAE的解码器随机生成一张图片
with torch.no_grad():
    sample = torch.randn(64, 20).to(device)
    sample = model.decode(sample).cpu()
    save_image(sample.view(64, 1, 28, 28),
               './results/sample.png')